# BERT Fine-Tuning Lab

이 노트북은 **IMDB 영화 리뷰 데이터셋**을 사용하여 긍정/부정을 분류하는 모델을 만드는 과정을 담고 있습니다.
사전 학습된 `distilbert-base-uncased` 모델을 가져와 파인튜닝(Fine-tuning)을 진행합니다.

## 1. 라이브러리 설치

Hugging Face의 `transformers`와 `datasets` 라이브러리를 설치합니다. 
이전 버전 호환성을 위해 `datasets`와 `fsspec` 버전을 특정합니다.

In [1]:
# 세션 다시 시작 필요 (코랩 환경일 경우 설치 후 런타임 재시작 권장)
!pip install -q transformers datasets
!pip install -U "datasets<=2.18.0" "fsspec<=2023.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.6.0 which is incompatible.


## 2. 데이터 로드 및 분할

IMDB 영화 리뷰 데이터셋을 불러옵니다. 빠른 실습을 위해 **50개의 샘플**만 사용합니다.

In [2]:
from datasets import load_dataset

# 1. 데이터 로드
# IMDB 영화 리뷰 데이터셋 중 50개 샘플만 가져와서 학습용/평가용으로 8:2 비율로 나눕니다
dataset = load_dataset("imdb", split="train[:50]").train_test_split(test_size=0.2)

# 데이터 확인
print("데이터셋 구조:", dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

데이터셋 구조: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 40
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
})


In [3]:
# 샘플 데이터 확인 (인덱스 7)
sample = dataset["train"][7]
print(f" 리뷰 내용:\n{sample['text']}\n")
print(f" 라벨 (0=부정, 1=긍정): {sample['label']}")

 리뷰 내용:
This movie sucked. It really was a waste of my life. The acting was atrocious, the plot completely implausible. Long, long story short, these people get "terrorized" by this pathetic "crazed killer", but completely fail to fight back in any manner. And this is after they take a raft on a camping trip, with no gear, and show up at a campsite that is already assembled and completely stocked with food and clothes and the daughters headphones. Additionally, after their boat goes missing, they panic that they're stuck in the woods, but then the daughters boyfriend just shows up and they apparently never consider that they could just hike out of the woods like he did to get to them. Like I said, this movie sucks. A complete joke. Don't let your girlfriend talk you into watching it.

 라벨 (0=부정, 1=긍정): 0


## 3. 모델 및 토크나이저 준비

`distilbert-base-uncased` 모델과 토크나이저를 불러옵니다.

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 2. 모델 및 토크나이저 준비
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# uncased: 대소문자 구분 없이 모두 소문자로 처리한다는 의미

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
# SequenceClassification: 문장 분류를 위한 헤드(Head)가 달린 모델을 불러옴

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 4. 데이터 전처리 (토큰화)

문장을 모델이 이해할 수 있는 숫자(토큰 ID)로 변환합니다.

In [5]:
# 3. 데이터 전처리 (텍스트를 숫자 토큰으로 변환)
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## 5. 훈련 설정 (Trainer)

`Trainer`와 `TrainingArguments`를 설정하여 학습을 준비합니다.

In [6]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score

# 정확도 계산 함수: 모델의 예측값과 실제 정답을 비교하여 정확도를 계산
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1) # 확률이 가장 높은 클래스 선택
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

# 훈련 파라미터 설정
args = TrainingArguments(
    output_dir="test",              # 모델 체크포인트 저장 경로
    per_device_train_batch_size=8,  # 배치 크기 (한 번에 학습할 데이터 수)
    num_train_epochs=15,            # 학습 반복 횟수 (전체 데이터를 15번 봄)
    report_to="none",               # 외부 로깅 비활성화
    logging_steps=1                 # 매 스텝마다 로그 출력
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,                    # 학습시킬 모델
    args=args,                      # 위에서 정의한 설정값
    train_dataset=dataset["train"], # 학습 데이터 (40개)
    eval_dataset=dataset["test"],   # 평가 데이터 (10개)
    compute_metrics=compute_metrics # 평가 방식 (정확도)
)

## 6. 파인튜닝 실행 및 평가

모델 학습을 시작하고, 테스트 데이터로 성능을 평가합니다.

In [7]:
# 5. 파인튜닝 실행 (모델 학습)
trainer.train()

Step,Training Loss
1,0.610400
2,0.459200
3,0.330100
4,0.210800
5,0.153200
6,0.128900
7,0.076600
8,0.048500
9,0.031600
10,0.032800


TrainOutput(global_step=75, training_loss=0.03038031229361271, metrics={'train_runtime': 34.3887, 'train_samples_per_second': 17.448, 'train_steps_per_second': 2.181, 'total_flos': 79480439193600.0, 'train_loss': 0.03038031229361271, 'epoch': 15.0})

In [8]:
# 6. 모델 평가
results = trainer.evaluate()
print(results)

{'eval_loss': 0.0008765311213210225, 'eval_accuracy': 1.0, 'eval_runtime': 0.2332, 'eval_samples_per_second': 42.877, 'eval_steps_per_second': 8.575, 'epoch': 15.0}


## 7. 실제 예측 수행 (Inference)

학습된 모델을 사용하여 새로운 문장의 감정을 예측합니다.

In [9]:
import torch

# 6. 학습된 모델로 실제 예측 수행
text = "I would put this at the top of my list of films in the category of unwatchable trash!"
# "이 영화를 내 '시청 불가 쓰레기' 카테고리 영화 리스트의 맨 위에 올리겠다!" (혹평)

# 1. 토큰화 및 장치 이동
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

inputs = tokenizer(text, return_tensors="pt").to(device)

# 2. 모델 예측
with torch.no_grad():
    output = model(**inputs)

# 3. 결과 해석 (Logits -> Label)
label = output.logits.argmax(-1).item()
# output.logits는 [부정점수, 긍정점수] 형태입니다.
# argmax(-1)은 둘 중 더 큰 점수의 위치(인덱스)를 찾습니다.

print(f"입력 문장: {text}")
print("예측 결과:", "긍정" if label == 1 else "부정")

입력 문장: I would put this at the top of my list of films in the category of unwatchable trash!
예측 결과: 부정
